In [9]:
"""
In this script the kalman filter EM algorithm is applied to the dataset
"""

# import necessary libraries
import sys
import numpy as np

sys.path.insert(0,'/Users/ashwin/Documents/Y4 project Brain Human Interfaces/General 4th year Github repo/BrainMachineInterfaces')
from Experiment_pointer.DataAnalysis.postProcessRawData import processTrialData, processMulipleTrialData

DOFOffset = 0.03

testData = processMulipleTrialData(['23_11_ashTrial1_90s.npz'], '23_11_trial1_cal_matrix.npz',DOFOffset)

trainingData = processMulipleTrialData(['23_11_ashTrial2_120s.npz','23_11_ashTrial3_120s.npz','23_11_ashTrial4_120s.npz','23_11_ashTrial5_120s.npz'], '23_11_trial1_cal_matrix.npz',DOFOffset)
print('program ended')

program ended


In [13]:
#print([saccades['rigidBodyData'][i].shape[1] for i in range(0,len(saccades['goCues']))])
print(len(testData['timestamps']))


19


In [14]:
# let's have 8 latents for now
# initialise at random states
# mu_1 =  np.zeros((8,1))
# sigma_1 = np.zeros((8,8))
# A = np.zeros((8,8))
# C = np.zeros((2,8))
# Q = np.zeros((8,8))
# R = np.zeros((2,2))


In [36]:
z = trainingData['rigidBodyData'][1]
v = trainingData['cursorPosData'][1]

In [159]:
class KalmanFilter():
    def __init__(self,z,v):
        # z should be of shape N x T
        # v should be of shape 2 x T
        self.l = 8
        self.v = v
        self.z = z
        self.mu_1 =  np.ones((self.l,1)) * 0.1
        self.sigma_1 = np.identity(self.l) * 0.1
        self.A = np.ones((self.l,self.l)) * 0.2
        self.C = np.zeros((2,self.l)) 
        self.C[0,0] = 0.1
        self.C[1,1] = 0.1
        self.Q = np.identity(self.l) * 0.1
        self.R = np.identity(2) * 0.2
        self.T = len(self.v[0,:])
        
    def runFilterAndSmoother(self):
        # kalman filter first
        self.X = np.zeros((self.T,self.l))
        self.V = np.zeros((self.T,self.l,self.l))

        self.X_prev = self.mu_1
        self.V_prev = self.sigma_1

        for i in range(0,self.T):
            #
            self.X_curr = np.matmul(self.A,self.X_prev)
            
            self.V_curr = np.matmul(self.A,np.matmul(self.V_prev,self.A.T)) + self.Q 
            self.K_curr = np.matmul(np.matmul(self.V_curr,self.C.T), np.linalg.inv(np.matmul(self.C,np.matmul(self.V_curr,self.C.T)) + self.R))
            self.X_curr = self.X_curr.reshape(-1) + np.matmul(self.K_curr,self.v[:,i].reshape(-1) - np.matmul(self.C,self.X_prev).reshape(-1))
            #print(self.X_curr.shape)
            self.V_curr = self.V_curr - np.matmul(self.K_curr,np.matmul(self.C,self.V_prev))
             
            print(self.V_curr)

            self.X[i,:] = self.X_curr
            self.V[i,:,:] = self.V_curr

            self.X_prev = self.X_curr
            self.V_prev = self.V_curr
        
        # kalman smoother next
        self.V_KS = np.zeros((self.T,self.l,self.l))
        self.V_covariances = np.zeros((self.T-1,self.l,self.l))
        self.X_KS = np.zeros((self.T,self.l))
        self.X_KS[-1,:] = self.X_curr
        self.V_KS[-1,:,:] = self.V_curr
        self.J_KS = np.zeros((self.T,self.l,self.l))
        self.P = np.zeros((self.T,8,8))
        self.crossP = np.zeros((self.T - 1,8,8))
        self.P_curr = self.V_KS[-1,:,:] + np.matmul(self.X_KS[-1],self.X_KS[-1].T)
        self.P[-1,:,:] = self.P_curr
        self.V_cv_curr = np.matmul(np.identity(8) - np.matmul(self.K_curr,self.C),np.matmul(self.A,self.V[-2,:,:]))
        self.V_covariances[-1,:,:] = self.V_cv_curr
        
        
        for j in range(self.T - 1, 0):
            self.J_curr = np.matmul(self.V[j,:,:],np.matmul(self.A.T,np.linalg.inv(self.V[j+1,:,:])))
            self.X_curr = self.X[j,:] + np.matmul(self.J_curr,self.X_KS[j+1,:] - np.matmul(self.A,self.X[j,:]))
            self.V_curr = self.V[j,:] + np.matmul(self.J_curr,np.matmul(self.V_KS[j+1] - self.V[j], self.J_curr.T) )
            for j in range(self.T - 2, 0):
                self.V_cv_curr = np.matmul(self.V[j,:,:],self.J_prev.T) + np.matmul(self.J_curr,np.matmul(self.V_covariances[j+1] - np.matmul(self.A,self.V[j,:,:]),self.J_prev))
            self.currCrossP = self.V_cv_curr + np.matmul(self.X_KS[j+1,:,:],self.X_curr.T)        
            self.crossP[j,:,:] = self.currCrossP                     
            self.J_prev = self.J_curr
            self.X_KS[j,:] = self.X_curr
            self.V_KS[j,:,:] = self.V_curr
            self.V_covariances[j,:,:] = self.V_cv_curr

            self.P_curr = self.V_KS[j,:,:] + np.matmul(self.X_KS[j],self.X_KS[j])
            self.P[j,:,:] = self.P_curr   

            
            self.crossP[j,:,:]  = self.currCrossP
    def reEstimateParameters(self):
        self.mu_1 = self.X_KS[0,:]
        self.sigma_1 = self.P[0,:,:] - np.matmul(self.X_KS[0],self.X_KS[0].T)
        
        # update C
        C_new_i = np.zeros((2,self.l))
        C_new_ii = np.zeros((self.l,self.l))
        for i in range(0,self.T):
            C_new_i += np.matmul(self.v[:,i].reshape(2,1),self.X_KS[i,:].reshape(8,1).T) # not sure if transpose missed
            C_new_ii += self.P[i,:,:]
        C_new = np.matmul(C_new_i,C_new_ii)
        self.C = C_new
        
        # update R
        term1 = np.zeros((2,2))

        for i in range(0,self.T):
            term1 += np.matmul(self.v[:,i].reshape(2,1),self.v[:,i].reshape(2,1).T) - np.matmul(self.C,np.matmul(self.X_KS[i,:].reshape(8,1),self.v[:,i].reshape(2,1).T))
        R_new = term1/self.T 
        self.R = R_new

        # update A
        term1 = np.zeros((8,8))
        term2 = np.zeros(term1.shape)
        for i in range(0,self.T-1):
            term1 += self.crossP[i,:,:]
            term2 = term2 + self.P[i,:,:]
        try:
            term2 = np.linalg.inv(term2)
        except:
            print("EXCEPTION RAISED")
            term2 = np.identity(self.l)
        A_new = np.matmul(term1,term2)
        #print(A_new)
        self.A = A_new

        # update Q
        term1 = np.zeros((8,8))
        term2 = np.zeros((8,8))
        for i in range(0,self.T-1):
            term1 += self.P[i,:,:] 
            term2 += self.crossP[i,:,:]
        Q_new = term1 - np.matmul(self.A,term2)
        Q_new = Q_new/(self.T - 1)
        #print(Q_new)
        self.Q = Q_new


In [160]:
kf = KalmanFilter(z,v)
for i in range(0,10):
    kf.runFilterAndSmoother()
    kf.reEstimateParameters()

print(kf.A)


[[0.13134458 0.03184209 0.032      0.032      0.032      0.032
  0.032      0.032     ]
 [0.03184209 0.13134458 0.032      0.032      0.032      0.032
  0.032      0.032     ]
 [0.0318413  0.0318413  0.132      0.032      0.032      0.032
  0.032      0.032     ]
 [0.0318413  0.0318413  0.032      0.132      0.032      0.032
  0.032      0.032     ]
 [0.0318413  0.0318413  0.032      0.032      0.132      0.032
  0.032      0.032     ]
 [0.0318413  0.0318413  0.032      0.032      0.032      0.132
  0.032      0.032     ]
 [0.0318413  0.0318413  0.032      0.032      0.032      0.032
  0.132      0.032     ]
 [0.0318413  0.0318413  0.032      0.032      0.032      0.032
  0.032      0.132     ]]
[[0.21221645 0.11271148 0.11326311 0.11326311 0.11326311 0.11326311
  0.11326311 0.11326311]
 [0.11271148 0.21221645 0.11326311 0.11326311 0.11326311 0.11326311
  0.11326311 0.11326311]
 [0.11286536 0.11286536 0.21342053 0.11342053 0.11342053 0.11342053
  0.11342053 0.11342053]
 [0.11286536 0.1